In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import glob
from epiweeks import Week
from metrics import *
EPS = 1e-6
import matplotlib.pyplot as plt
import math

In [2]:
# ground truth
df_ground_truth = pd.read_csv('./data-truth/truth-Incident Deaths.csv') 

/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6647/441569599.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ground_truth = pd.read_csv('./data-truth/truth-Incident Deaths.csv')


In [3]:
df_ground_truth.head()
df_grnd = df_ground_truth[['date', 'location', 'location_name', 'value']]
df_grnd = df_grnd[df_grnd['date']>='2022-01-01']
df_grnd = df_grnd[df_grnd['location_name'].str.contains("County") == False]
df_grnd['date'] = df_grnd['date'].apply(lambda x: Week.fromdate(datetime.strptime(x, '%Y-%m-%d')))
df_grnd['date'] = df_grnd['date'].astype(str)
df_grnd = df_grnd[df_grnd['date'] >= '202222']
df_grnd = df_grnd.groupby(['date', 'location', 'location_name'], as_index= False).sum()
df_grnd = df_grnd.sort_values('location_name', kind = 'mergesort')
df_grnd = df_grnd.rename(
    columns = {'date':"predicted_week"}
)
df_grnd

,predicted_week,location,location_name,value
0,202222,1,Alabama,6
57,202223,1,Alabama,28
114,202224,1,Alabama,4
171,202225,1,Alabama,33
228,202226,1,Alabama,30
...,...,...,...,...
564,202231,56,Wyoming,0
621,202232,56,Wyoming,9
678,202233,56,Wyoming,6
735,202234,56,Wyoming,6


In [4]:
file_dir = './predictions.csv' 
df_total = pd.read_csv(file_dir)

/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6647/4166721719.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_total = pd.read_csv(file_dir)


In [5]:
len(df_total)

2173183

In [6]:
df_total['model'].nunique()
df_final = df_total.copy()
all_model_names = np.array(df_final['model'].drop_duplicates())
all_model_names

array(['BPagano-RtDriven', 'CEID-Walk', 'COVIDhub-4_week_ensemble',
       'COVIDhub-baseline', 'COVIDhub-ensemble',
       'COVIDhub-trained_ensemble', 'COVIDhub_CDC-ensemble',
       'CU-nochange', 'CU-scenario_low', 'CU-scenario_mid', 'CU-select',
       'CovidAnalytics-DELPHI', 'GT-DeepCOVID', 'IHME-CurveFit',
       'JHUAPL-Bucky', 'JHU_IDD-CovidSP', 'KITmetricslab-select_ensemble',
       'Karlen-pypm', 'LUcompUncertLab-VAR_3streams', 'MIT-Cassandra',
       'MIT_ISOLAT-Mixtures', 'MOBS-GLEAM_COVID', 'MUNI-ARIMA',
       'Microsoft-DeepSTIA', 'PSI-DRAFT', 'QJHong-Encounter',
       'RobertWalraven-ESG', 'SDSC_ISG-TrendModel',
       'SteveMcConnell-CovidComplete', 'TTU-squider',
       'UCSD_NEU-DeepGLEAM', 'USC-SI_kJalpha'], dtype=object)

In [7]:
all_model_names = np.array(df_final['model'].drop_duplicates())
df_gt = df_final[df_final['model']=='GT-DeepCOVID']
# GT-FluFNP model hasn't predicted for some locations 
all_regions = np.array(df_gt['location'].drop_duplicates())
print(all_regions)
regions_ground_truth = np.array(df_grnd['location'].drop_duplicates())
regions_ground_truth

['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'IA' 'ID' 'IL'
 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN'
 'TX' 'US' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


array([1, 2, 60, 4, 5, 6, 8, 9, 10, 11, '12', '13', '66', '15', '16',
       '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
       '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '69', '39', '40', '41', '42', '72', '44', '45', '46', '47', '48',
       'US', '49', '50', '78', '51', '53', '54', '55', '56'], dtype=object)

In [8]:
df_point = df_final[df_final['type']=='point']
df_quant = df_final[df_final['type']=='quantile']

In [9]:
weeks = np.array(df_point['forecast_week'].drop_duplicates())
max_week = df_grnd['predicted_week'].max()

In [10]:
df_point['predicted_week'] = df_point['forecast_week']+df_point['ahead']
df_point['predicted_week'] = df_point['predicted_week'].astype(str)
df_point = df_point[df_point['predicted_week']<=max_week] 
df_point

/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6647/2619530944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_point['predicted_week'] = df_point['forecast_week']+df_point['ahead']
/var/folders/8d/yc9nnvjx07j3ds1jzwshhn580000gn/T/ipykernel_6647/2619530944.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_point['predicted_week'] = df_point['predicted_week'].astype(str)


,model,forecast_week,ahead,location,type,quantile,value,predicted_week
0,BPagano-RtDriven,202223,1,AK,point,NaN,6.012000,202224
24,BPagano-RtDriven,202223,2,AK,point,NaN,6.069000,202225
48,BPagano-RtDriven,202223,3,AK,point,NaN,6.198000,202226
72,BPagano-RtDriven,202223,4,AK,point,NaN,6.398000,202227
96,BPagano-RtDriven,202223,1,AL,point,NaN,26.006000,202224
...,...,...,...,...,...,...,...,...
2163607,USC-SI_kJalpha,202234,1,VT,point,NaN,4.485173,202235
2163775,USC-SI_kJalpha,202234,1,WA,point,NaN,73.635732,202235
2163943,USC-SI_kJalpha,202234,1,WI,point,NaN,36.373721,202235
2164111,USC-SI_kJalpha,202234,1,WV,point,NaN,34.665374,202235


In [11]:
location_df = pd.read_csv('data-locations/locations.csv')
location_dict = {location_df['location_name'][i]:location_df['abbreviation'][i] for i in range(len(location_df)) if not pd.isnull(location_df['abbreviation'][i])}
location_dict['United States'] = 'US'
location_dict

{'US': 'US',
 'Alabama': 'AL',
 'Alaska': 'AK',
 'Arizona': 'AZ',
 'Arkansas': 'AR',
 'California': 'CA',
 'Colorado': 'CO',
 'Connecticut': 'CT',
 'Delaware': 'DE',
 'District of Columbia': 'DC',
 'Florida': 'FL',
 'Georgia': 'GA',
 'Hawaii': 'HI',
 'Idaho': 'ID',
 'Illinois': 'IL',
 'Indiana': 'IN',
 'Iowa': 'IA',
 'Kansas': 'KS',
 'Kentucky': 'KY',
 'Louisiana': 'LA',
 'Maine': 'ME',
 'Maryland': 'MD',
 'Massachusetts': 'MA',
 'Michigan': 'MI',
 'Minnesota': 'MN',
 'Mississippi': 'MS',
 'Missouri': 'MO',
 'Montana': 'MT',
 'Nebraska': 'NE',
 'Nevada': 'NV',
 'New Hampshire': 'NH',
 'New Jersey': 'NJ',
 'New Mexico': 'NM',
 'New York': 'NY',
 'North Carolina': 'NC',
 'North Dakota': 'ND',
 'Ohio': 'OH',
 'Oklahoma': 'OK',
 'Oregon': 'OR',
 'Pennsylvania': 'PA',
 'Rhode Island': 'RI',
 'South Carolina': 'SC',
 'South Dakota': 'SD',
 'Tennessee': 'TN',
 'Texas': 'TX',
 'Utah': 'UT',
 'Vermont': 'VT',
 'Virginia': 'VA',
 'Washington': 'WA',
 'West Virginia': 'WV',
 'Wisconsin': 'WI',
 '

In [12]:
df_grnd['location_name'] = df_grnd['location_name'].apply(lambda x: location_dict[x])
df_grnd

,predicted_week,location,location_name,value
0,202222,1,AL,6
57,202223,1,AL,28
114,202224,1,AL,4
171,202225,1,AL,33
228,202226,1,AL,30
...,...,...,...,...
564,202231,56,WY,0
621,202232,56,WY,9
678,202233,56,WY,6
735,202234,56,WY,6


In [13]:
df_newpoint = pd.merge(df_point, df_grnd, on = "predicted_week")
df_newpoint = df_newpoint[df_newpoint['location_x'] == df_newpoint['location_name']]
df_newpoint

,model,forecast_week,ahead,location_x,type,quantile,value_x,predicted_week,location_y,location_name,value_y
1,BPagano-RtDriven,202223,1,AK,point,NaN,6.012,202224,2,AK,0
57,BPagano-RtDriven,202223,1,AL,point,NaN,26.006,202224,1,AL,4
118,BPagano-RtDriven,202223,1,AR,point,NaN,24.906,202224,5,AR,16
174,BPagano-RtDriven,202223,1,AZ,point,NaN,43.822,202224,4,AZ,28
233,BPagano-RtDriven,202223,1,CA,point,NaN,246.373,202224,6,CA,225
...,...,...,...,...,...,...,...,...,...,...,...
4125026,SteveMcConnell-CovidComplete,202222,1,VT,point,NaN,4.000,202223,50,VT,3
4125086,SteveMcConnell-CovidComplete,202222,1,WA,point,NaN,57.000,202223,53,WA,78
4125145,SteveMcConnell-CovidComplete,202222,1,WI,point,NaN,47.000,202223,55,WI,45
4125201,SteveMcConnell-CovidComplete,202222,1,WV,point,NaN,32.000,202223,54,WV,25


In [14]:
for i in df_newpoint['model'].unique(): 
    print(i)
    dl = df_newpoint[df_newpoint['model'] == i]
    print(dl['ahead'].unique())

BPagano-RtDriven
[1 2 3 4]
CEID-Walk
[2 1 3 4]
COVIDhub-4_week_ensemble
[2 1 3 4]
COVIDhub-baseline
[2 1 3 4 5 6 7 8]
COVIDhub-ensemble
[2 1 3 4]
COVIDhub-trained_ensemble
[2 1 3 4]
COVIDhub_CDC-ensemble
[2 1 3 4]
CU-nochange
[1 2 3 4]
CU-scenario_low
[1 2 3 4]
CU-scenario_mid
[1 2 3 4]
CU-select
[1 2 3 4]
CovidAnalytics-DELPHI
[2 1 3 4 5 6]
GT-DeepCOVID
[2 1 3 4]
IHME-CurveFit
[1 2 3 4 5 6 7 8 9]
JHUAPL-Bucky
[2 1 3 4 5 6 7]
JHU_IDD-CovidSP
[1 2 3 4 5 6 7 8 9]
KITmetricslab-select_ensemble
[2 1 3 4]
Karlen-pypm
[1 2 3 4 5]
MIT-Cassandra
[2 1 3 4 5]
MIT_ISOLAT-Mixtures
[2 1 3 4]
MOBS-GLEAM_COVID
[1 2 3 4]
MUNI-ARIMA
[2 1 3 4]
Microsoft-DeepSTIA
[2 1 3 4]
PSI-DRAFT
[2 1 3 4]
QJHong-Encounter
[1 2 3 4 5 6 7 8]
RobertWalraven-ESG
[1 2 3 4]
SDSC_ISG-TrendModel
[1]
SteveMcConnell-CovidComplete
[2 1 3 4]
TTU-squider
[1 2 3 4]
UCSD_NEU-DeepGLEAM
[1 2 3 4]
USC-SI_kJalpha
[1 2 3 4 5 6 7]


In [39]:
rmse_all = []
nrmse_all = []
model_all= []
mape_all = []
week_ahead = []
regions = []

In [40]:
model_with_issue = ['MIT_ISOLAT-Mixtures','TTU-squider','JHU_IDD-CovidSP','LUcompUncertLab-VAR_3streams','SDSC_ISG-TrendModel']
for model in all_model_names:
    print(model)
    if model in model_with_issue:
        continue
    for i in range(1, 5):
        for region in all_regions:
            sample = df_newpoint[
                (df_newpoint["model"] == model)
                & (df_newpoint["ahead"] == i)
                & (df_newpoint["location_x"] == region)
            ]["value_x"].values
            target = df_newpoint[
                (df_newpoint["model"] == model)
                & (df_newpoint["ahead"] == i)
                & (df_newpoint["location_x"] == region)
            ]["value_y"].values
            rmse_all.append(rmse(sample, target))
            nrmse_all.append(norm_rmse(sample, target))

            #             Deal with inf values
            target = np.array([EPS if x == 0 else x for x in target]).reshape(
                (len(target), 1)
            )
            mape_all.append(mape(sample, target))
            model_all.append(model)
            week_ahead.append(i)
            regions.append(region)

BPagano-RtDriven
CEID-Walk
COVIDhub-4_week_ensemble
COVIDhub-baseline
COVIDhub-ensemble
COVIDhub-trained_ensemble
COVIDhub_CDC-ensemble
CU-nochange
CU-scenario_low
CU-scenario_mid
CU-select
CovidAnalytics-DELPHI
GT-DeepCOVID
IHME-CurveFit
JHUAPL-Bucky
JHU_IDD-CovidSP
KITmetricslab-select_ensemble
Karlen-pypm
LUcompUncertLab-VAR_3streams
MIT-Cassandra
MIT_ISOLAT-Mixtures
MOBS-GLEAM_COVID
MUNI-ARIMA
Microsoft-DeepSTIA
PSI-DRAFT
QJHong-Encounter
RobertWalraven-ESG
SDSC_ISG-TrendModel
SteveMcConnell-CovidComplete
TTU-squider
UCSD_NEU-DeepGLEAM
USC-SI_kJalpha


In [54]:
print(len(model_all))
print(len(week_ahead))
print(len(regions))
print(len(rmse_all))
print(len(nrmse_all))
print(len(mape_all))

11016
11016
11016
5508
5508
5508


In [43]:
df_point_scores = pd.DataFrame.from_dict(
    {
        "Model": model_all,
        "Weeks ahead": week_ahead,
        "Location": regions,
        "RMSE": rmse_all,
        "NRMSE": nrmse_all,
        "MAPE": mape_all,
    }
)
df_point_scores.tail(50)

,Model,Weeks ahead,Location,RMSE,NRMSE,MAPE
5458,USC-SI_kJalpha,4,AL,44.867725,0.590365,1.970497e+02
5459,USC-SI_kJalpha,4,AR,85.442792,2.136070,2.039533e+02
5460,USC-SI_kJalpha,4,AZ,36.092535,1.503856,4.158942e+01
5461,USC-SI_kJalpha,4,CA,196.228334,1.521150,4.114444e+01
5462,USC-SI_kJalpha,4,CO,16.591874,0.338610,1.295486e+02
5463,USC-SI_kJalpha,4,CT,25.754696,0.402417,9.995277e+01
5464,USC-SI_kJalpha,4,DC,3.363840,0.420480,6.799036e+01
5465,USC-SI_kJalpha,4,DE,19.176979,0.737576,2.983685e+02
5466,USC-SI_kJalpha,4,FL,140.284156,0.552300,2.807434e+01
5467,USC-SI_kJalpha,4,GA,208.057855,3.852923,1.046848e+02


In [44]:
df_point_scores.to_csv('point_scores.csv')

In [20]:
df_quant = df_final[df_final['type']=='quantile']
df_quant

,model,forecast_week,ahead,location,type,quantile,value
1,BPagano-RtDriven,202223,1,AK,quantile,0.01,0.000010
2,BPagano-RtDriven,202223,1,AK,quantile,0.025,0.582640
3,BPagano-RtDriven,202223,1,AK,quantile,0.05,1.431960
4,BPagano-RtDriven,202223,1,AK,quantile,0.1,2.426480
5,BPagano-RtDriven,202223,1,AK,quantile,0.15,3.101850
...,...,...,...,...,...,...,...
2173178,USC-SI_kJalpha,202235,7,WY,quantile,0.85,1.513421
2173179,USC-SI_kJalpha,202235,7,WY,quantile,0.9,1.525637
2173180,USC-SI_kJalpha,202235,7,WY,quantile,0.95,2.075220
2173181,USC-SI_kJalpha,202235,7,WY,quantile,0.975,4.512720


In [21]:
norm_df_quant = df_quant.copy()
norm_df_quant['predicted_week'] = (
    norm_df_quant['forecast_week']+norm_df_quant['ahead']
)
norm_df_quant['predicted_week'] = norm_df_quant['predicted_week'].astype(str)
norm_df_quant = norm_df_quant[norm_df_quant['predicted_week']<=max_week] 
norm_df_quant

,model,forecast_week,ahead,location,type,quantile,value,predicted_week
1,BPagano-RtDriven,202223,1,AK,quantile,0.01,0.000010,202224
2,BPagano-RtDriven,202223,1,AK,quantile,0.025,0.582640,202224
3,BPagano-RtDriven,202223,1,AK,quantile,0.05,1.431960,202224
4,BPagano-RtDriven,202223,1,AK,quantile,0.1,2.426480,202224
5,BPagano-RtDriven,202223,1,AK,quantile,0.15,3.101850,202224
...,...,...,...,...,...,...,...,...
2164298,USC-SI_kJalpha,202234,1,WY,quantile,0.85,10.013516,202235
2164299,USC-SI_kJalpha,202234,1,WY,quantile,0.9,10.509538,202235
2164300,USC-SI_kJalpha,202234,1,WY,quantile,0.95,10.964335,202235
2164301,USC-SI_kJalpha,202234,1,WY,quantile,0.975,12.823314,202235


In [22]:
for i in norm_df_quant['model'].unique(): 
    print(i)
    dl = norm_df_quant[norm_df_quant['model'] == i]
    print(dl['ahead'].unique())

BPagano-RtDriven
[1 2 3 4]
CEID-Walk
[1 2 3 4]
COVIDhub-4_week_ensemble
[1 2 3 4]
COVIDhub-baseline
[1 2 3 4 5 6 7 8]
COVIDhub-ensemble
[1 2 3 4]
COVIDhub-trained_ensemble
[1 2 3 4]
COVIDhub_CDC-ensemble
[1 2 3 4]
CU-nochange
[1 2 3 4]
CU-scenario_low
[1 2 3 4]
CU-scenario_mid
[1 2 3 4]
CU-select
[1 2 3 4]
CovidAnalytics-DELPHI
[1 2 3 4 5 6]
GT-DeepCOVID
[1 2 3 4]
IHME-CurveFit
[1 2 3 4 5 6 7 8 9]
JHUAPL-Bucky
[1 2 3 4 5 6 7]
JHU_IDD-CovidSP
[1 2 3 4 5 6 7 8 9]
KITmetricslab-select_ensemble
[1 2 3 4]
Karlen-pypm
[1 2 3 4 5]
LUcompUncertLab-VAR_3streams
[1 2 3 4]
MIT-Cassandra
[1 2 3 4 5]
MIT_ISOLAT-Mixtures
[1 2 3 4]
MOBS-GLEAM_COVID
[1 2 3 4]
MUNI-ARIMA
[1 2 3 4]
Microsoft-DeepSTIA
[1 2 3 4]
PSI-DRAFT
[1 2 3 4]
QJHong-Encounter
[1 2 3 4 5 6 7 8]
RobertWalraven-ESG
[1 2 3 4]
SDSC_ISG-TrendModel
[1]
SteveMcConnell-CovidComplete
[1 2 3 4]
UCSD_NEU-DeepGLEAM
[1 2 3 4]
USC-SI_kJalpha
[1 2 3 4 5 6 7]


In [83]:
week_ahead = []
regions = []
crps_all = []
ls_all = []
model_all = []
cs_all = []

In [63]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
count = 0
model_with_issue = ['MIT_ISOLAT-Mixtures','TTU-squider','JHU_IDD-CovidSP','LUcompUncertLab-VAR_3streams','SDSC_ISG-TrendModel']
for k in range(len(all_model_names)):
    model = all_model_names[k]
    print('Compiling scores of model ', model)
    print(f"Model {count}/{len(all_model_names)}")
    count += 1
    if model in model_with_issue:
        continue
#     All Weeks ahead
    for i in range(1, 5):
        print('Week ahead ', i)
#         All regions
        for region in all_regions:
#             Dataset with information about Ground truth ('value_y') and predictions ('value_x') 
            target = df_newpoint
            target = target[(target['location_x']==region)]
            target = target[(target['model']== model)]
            if i == 1: 
                target =  target[(target['ahead'] == 1)]
            elif i == 2:
                target = target[(target['ahead'] == 2)]
            elif i == 3:
                 target = target[(target['ahead'] == 3)]
            else:
                target = target[(target['ahead'] == 4)]
            norm_model = norm_df_quant[ 
                                (norm_df_quant['model']==model)
                                & (norm_df_quant['location']==region)
            ]
            if i == 1: 
                norm_model =  norm_model[(norm_model['ahead'] == 1)]
            elif i == 2:
                norm_model = norm_model[(norm_model['ahead'] == 2)]
            elif i == 3:
                 norm_model = norm_model[(norm_model['ahead'] == 3)]
            else:
                norm_model = norm_model[(norm_model['ahead'] == 4)]
            mean_ = []
            std_ = []
            var_ = []
            tg_vals = []
            pred_vals = []
            weeks = np.array(target['forecast_week'].drop_duplicates())
            if(len(weeks)!=0):
                for week in weeks:
    #                 Append point predictions
                    point_val = target[ (target['forecast_week']==week)][
                        'value_x'
                    ].values
                    mean_.append(point_val)
    #                 Append point pred as predictions
                    predval = target[ (target['forecast_week']==week)][
                        'value_y'
                    ].values 
                    pred_vals.append(predval)
                
    #                     Append ground truth as target
                    tgval = target[ (target['forecast_week']==week)]['value_y'].values
                    tg_vals.append(tgval)

    #                 Find std from quantiles
                    b = norm_model[  
                                    (norm_model['forecast_week']==week) 
                                    & (norm_model['quantile']==0.75)
                    ]['value'].values
                    a = norm_model[  
                                    (norm_model['forecast_week']==week)
                                    & (norm_model['quantile']==0.25)
                    ]['value'].values
                    std = (b-a)/1.35
                
                    var = std**2
                    std_.append(std)
                    var_.append(var)
                for j in range(len(std_)): 
                    if len(std_[j]) == 0:
                        std_[j] = [0]

                std_ = np.array(std_)
                var_ = np.array(var_)
                pred_vals = np.array(pred_vals)
                mean_ = np.array(mean_)
                tg_vals = np.array(tg_vals)

                if(len(tg_vals)==0):
                    print(
                        "No target found for week ahead ",
                        i, 
                        " region ",
                        region,
                        "model ",
                        model
                    )

#             Calculate ls and crps
                if(0 in std_):
                    cr_target = np.array([EPS if x ==0 else x for x in tg_vals], dtype=object).reshape(
                        (len(tg_vals), 1)
                    )
                    cr = float(mape(pred_vals, cr_target))
                    ls = -10
#                     print(cr, ls)
                else:
                    cr = crps(mean_, std_, tg_vals)
                    ls = log_score(mean_, std_, tg_vals, window = 0.1)
                    if(ls<-10):
                        ls = -10
#                     print(cr, ls, "hi")
                auc, cs, _ = get_pr(mean_, std_**2, tg_vals)

                
#                 if(ls<-10 or math.isnan(ls)):
#                     ls = -10
#                 elif(ls>10):
#                     ls = 10
#                 if(math.isnan(cr)):
#                     cr = 0
                    
                crps_all.append(cr)
                ls_all.append(ls)
                cs_all.append(cs)
            else:
                crps_all.append(np.nan)
                ls_all.append(np.nan)
                cs_all.append(np.nan)
            week_ahead.append(i)
            regions.append(region)
            model_all.append(model)

NameError: name 'all_model_names' is not defined

In [86]:
print(len(model_all))
print(len(week_ahead))
print(len(regions))
print(len(ls_all))
print(len(crps_all))
print(len(cs_all))


5508
5508
5508
5508
5508
5508


In [87]:
df_spread_scores = pd.DataFrame.from_dict(
    {
        'Model':model_all,
        'Weeks ahead':week_ahead,
        'Location':regions,
        'LS':ls_all,
        'CRPS':crps_all,
        'CS':cs_all
    }
)

In [88]:
df_spread_scores.to_csv('spread_scores.csv')